# Training a T5 model to generate Reverse English

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q transformers datasets pytorch-lightning sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 20.6 MB/s eta 0:00:00


# Load and process "ag-news" dataset



In [5]:
from datasets import load_dataset

dataset = load_dataset("ag_news")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


In [3]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")


max_input_length = 177
max_target_length = 200

def preprocess_examples(examples):
  # encode the english
  texts = examples['text']

  inputs = [' '.join(text.split()) for text in texts]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the reverse-english
  reverses = [' '.join(text[::-1].split()) for text in texts]
  labels = tokenizer(reverses, max_length=max_target_length, padding="max_length", truncation=True).input_ids


  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)

  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
dataset = dataset.map(preprocess_examples, batched=True)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7600
    })
})

In [8]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
idx = int(len(dataset['train'])*0.7)
train = [dataset['train'][i] for i in range(len(dataset['train'])) if i < idx]
validation = [dataset['train'][i] for i in range(len(dataset['train'])) if i > idx]

In [9]:
import os

os.cpu_count()

8

In [17]:
train_dataloader = DataLoader(train, shuffle=True, batch_size=24, num_workers=8)
valid_dataloader = DataLoader(validation, batch_size=4, num_workers=8)
test_dataloader = DataLoader(dataset['test'], batch_size=24, num_workers=8)

### Visualize working tokenizer

In [10]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [11]:
tokenizer.decode(batch['input_ids'][0])

'#39;Google #39; Your Cluttered Computer Online search-engine leader Google is targeting the computer hard drive with software that promises to scour the clutter of documents, e-mails, instant messages and other stored files.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [12]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

'.selif derots rehto dna segassem tnatsni,sliam-e,stnemucod fo rettulc eht ruocs ot sesimorp taht erawtfos htiw evird drah retupmoc eht gnitegrat si elgooG redael enigne-hcraes enilnO retupmoC derettulC ruoY ;93# elgooG;93#</s>'

# Define model class with pytorch_lightning

In [13]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class ReverseT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=15, warmup_steps=1000):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained("t5-small")
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs

    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss

    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)

        return loss

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)

        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}

        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [32]:
model = ReverseT5()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [14]:
# If restarting training ONLY
save_directory = "/content/drive/MyDrive/Reverse-English/Checkpoints-3/pretrained-epoch=09.ckpt"
model = ReverseT5.load_from_checkpoint(save_directory)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Training step

In [15]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint

early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=3,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

checkpoint_callback = ModelCheckpoint(
    dirpath="/content/drive/MyDrive/Reverse-English/Checkpoints-4",
    filename="pretrained-{epoch:02d}",
    save_top_k=-1,
    every_n_epochs=5  # Save every 5 epochs
)

trainer = Trainer(devices=1,
                  default_root_dir="/content/drive/MyDrive/Reverse-English/Checkpoints-4",
                  callbacks=[early_stop_callback, lr_monitor, checkpoint_callback],
                  max_epochs=10,
                )

trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /content/drive/MyDrive/Reverse-English/Checkpoints-4 exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Par

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [16]:
save_directory = "/content/drive/MyDrive/Reverse-English/Model-4"
model.model.save_pretrained(save_directory)

# Inference

In [11]:
from transformers import T5ForConditionalGeneration

save_directory = "/content/drive/MyDrive/Reverse-English/Model-4"
model = T5ForConditionalGeneration.from_pretrained(save_directory)

In [12]:
dataset_og = load_dataset("ag_news")
print(dataset_og['test'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset({
    features: ['text', 'label'],
    num_rows: 7600
})


In [13]:
for i in range(10):
    test_example = dataset_og['test'][i]
    print("Test:", test_example['text'])
    input_ids = tokenizer(test_example['text'], return_tensors='pt').input_ids

    # generate
    outputs = model.generate(input_ids, max_length=200)
    print("Reversed Sentence:", tokenizer.decode(outputs[0], skip_special_tokens=True))
    print()

Test: Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.
Reversed Sentence: .lugoM laredeF mrif tnerap nekcirts htiw sklat retfa 'detnioppasid' era yeht yas llaweN rehtorT ta srekrow gnitneserper snoinU sklat retfa noisnep N T rof sraeF

Test: The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the  #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket.
Reversed Sentence: .tekcor dennam sti rof etad hcnualtsrif eht decnuonna yllaiciffo sah,thgilf ecaps latibrobus dednuf yletavirprof tsetnoc a,ezirP X irasnA noillim 01;63# eht rof gnitepmoc sreetekcor fo maetdnoces A -- adanaC,OTNOROT - moc.ECAPS )moc.ECAPS( thgilfecapS namuH rof etaD hcnuaL steS maeT etavirP d

Test: Ky.

### Upon inspection, the above the sentences seem to be reversed fairly accurately. As a result, I saved all of the test inferences in a csv file.

In [32]:
import pandas as pd
import torch


inputs = []
inferences = []

model.eval()
i=0
with torch.no_grad():
    for batch in test_dataloader:
        if i%50==0:
            print(i)
        i+=1
        input_texts = batch['input_ids']

        outputs = model.generate(input_texts, max_length=200)
        output_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)

        inputs.extend(tokenizer.batch_decode(input_texts, skip_special_tokens=True))
        inferences+=output_texts

0
50
100
150
200
250
300


In [38]:
df = pd.DataFrame({'input': inputs, 'inference': inferences})

df.to_csv("/content/drive/MyDrive/Reverse-English/Result/inferences.csv", index=False)